In [1]:
%autosave 0

Autosave disabled


In [86]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import myfunctions
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler

In [3]:
base_url = '../../data/prepared/'

train = pd.read_csv(base_url + 'train_data.csv')
train_labels = pd.read_csv(base_url + 'train_labels.csv')

In [4]:
train.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827


In [5]:
categorical_columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [6]:
train[categorical_columns].isna().sum()

B_30        188
B_38        188
D_114     14565
D_116     14565
D_117     14565
D_120     14565
D_126      9642
D_63          0
D_64      18123
D_66     418967
D_68      17974
dtype: int64

## Nulls

Compute the null count for each column, categorical and continuous. Create a new feature from these null counts and scale from 0-1 so the range is appropriate for the rest of the dataset.

Fill in nulls in the categorical columns with 'None' so a dummy column is created for customers who have null values for a feature.

In [7]:
train_final = train.groupby('customer_ID').agg(lambda x: x.isnull().sum())

In [8]:
train_final.head()

,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0,0,0,0,0,0,0,0,0,13,...,13,13,13,0,0,0,13,0,0,0
0005ef1480ffe4f0e5908eae07db8e05f2e705bca6a57db3b97d1da2eb6c7cab,0,0,0,0,0,0,2,0,0,2,...,2,2,2,0,0,0,2,0,0,0
0005f2ab0cc352cd64eac9e60b981ed8c3289d20fa5fcef78973c468916fdfc9,0,0,0,0,0,0,0,0,0,13,...,13,13,13,0,0,0,13,0,0,0
0007ee1ac8edb6be588c6c8a158d2268e3228c49cf52773d168012076c118e5f,0,0,0,0,0,0,0,0,0,13,...,13,13,13,0,0,0,13,0,0,0
0008ef32824d3067b4d8ab70e7c79fa04c808f4d99194fe9c7827dac8bcf0b5c,0,0,0,0,0,0,13,0,0,13,...,13,13,13,1,0,1,13,1,0,1


In [9]:
train_final.shape

(39007, 189)

In [10]:
train_final.drop(columns='S_2', inplace=True)

In [11]:
train_final.columns = [x + '_nulls' for x in train_final.columns]
train_final.head()

,P_2_nulls,D_39_nulls,B_1_nulls,B_2_nulls,R_1_nulls,S_3_nulls,D_41_nulls,B_3_nulls,D_42_nulls,D_43_nulls,...,D_136_nulls,D_137_nulls,D_138_nulls,D_139_nulls,D_140_nulls,D_141_nulls,D_142_nulls,D_143_nulls,D_144_nulls,D_145_nulls
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0,0,0,0,0,0,0,0,13,13,...,13,13,13,0,0,0,13,0,0,0
0005ef1480ffe4f0e5908eae07db8e05f2e705bca6a57db3b97d1da2eb6c7cab,0,0,0,0,0,2,0,0,2,0,...,2,2,2,0,0,0,2,0,0,0
0005f2ab0cc352cd64eac9e60b981ed8c3289d20fa5fcef78973c468916fdfc9,0,0,0,0,0,0,0,0,13,4,...,13,13,13,0,0,0,13,0,0,0
0007ee1ac8edb6be588c6c8a158d2268e3228c49cf52773d168012076c118e5f,0,0,0,0,0,0,0,0,13,0,...,13,13,13,0,0,0,13,0,0,0
0008ef32824d3067b4d8ab70e7c79fa04c808f4d99194fe9c7827dac8bcf0b5c,0,0,0,0,0,13,0,0,13,0,...,13,13,13,1,0,1,13,1,0,1


In [14]:
mms = MinMaxScaler()

train_final[train_final.columns] = mms.fit_transform(train_final[train_final.columns])

train_final.head()

,P_2_nulls,D_39_nulls,B_1_nulls,B_2_nulls,R_1_nulls,S_3_nulls,D_41_nulls,B_3_nulls,D_42_nulls,D_43_nulls,...,D_136_nulls,D_137_nulls,D_138_nulls,D_139_nulls,D_140_nulls,D_141_nulls,D_142_nulls,D_143_nulls,D_144_nulls,D_145_nulls
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,1.000000,...,1.000000,1.000000,1.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000
0005ef1480ffe4f0e5908eae07db8e05f2e705bca6a57db3b97d1da2eb6c7cab,0.0,0.0,0.0,0.0,0.0,0.153846,0.0,0.0,0.153846,0.000000,...,0.153846,0.153846,0.153846,0.000000,0.0,0.000000,0.153846,0.000000,0.0,0.000000
0005f2ab0cc352cd64eac9e60b981ed8c3289d20fa5fcef78973c468916fdfc9,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,0.307692,...,1.000000,1.000000,1.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000
0007ee1ac8edb6be588c6c8a158d2268e3228c49cf52773d168012076c118e5f,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,0.000000,...,1.000000,1.000000,1.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000
0008ef32824d3067b4d8ab70e7c79fa04c808f4d99194fe9c7827dac8bcf0b5c,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,1.000000,0.000000,...,1.000000,1.000000,1.000000,0.076923,0.0,0.076923,1.000000,0.076923,0.0,0.076923


In [21]:
for col in categorical_columns:
    train[col].fillna('None', inplace=True)
    
train.head()    

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827


In [22]:
train['D_117'].value_counts()

-1.0    123351
3.0      99136
4.0      96897
2.0      56705
5.0      39870
6.0      29767
None     14565
1.0      10453
Name: D_117, dtype: int64

## Categorical features

Create a dataframe of the customer IDs, and concatenate the dummy dataframe to it. Take the aggregate last value for each categorical feature for each customer and merge it with the final train dataframe. 

In [28]:
dummy_df = pd.DataFrame(train['customer_ID'])
dummy_df.shape

(470744, 1)

In [29]:
dummies = pd.get_dummies(train[categorical_columns])

In [30]:
dummy_df = pd.concat([dummy_df, dummies], axis=1)
dummy_df.head()

,customer_ID,B_30_0.0,B_30_1.0,B_30_2.0,B_30_None,B_38_1.0,B_38_2.0,B_38_3.0,B_38_4.0,B_38_5.0,...,D_66_1.0,D_66_None,D_68_0.0,D_68_1.0,D_68_2.0,D_68_3.0,D_68_4.0,D_68_5.0,D_68_6.0,D_68_None
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [42]:
dummy_df.groupby('customer_ID').agg('last')

,B_30_0.0,B_30_1.0,B_30_2.0,B_30_None,B_38_1.0,B_38_2.0,B_38_3.0,B_38_4.0,B_38_5.0,B_38_6.0,...,D_66_1.0,D_66_None,D_68_0.0,D_68_1.0,D_68_2.0,D_68_3.0,D_68_4.0,D_68_5.0,D_68_6.0,D_68_None
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
0005ef1480ffe4f0e5908eae07db8e05f2e705bca6a57db3b97d1da2eb6c7cab,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
0005f2ab0cc352cd64eac9e60b981ed8c3289d20fa5fcef78973c468916fdfc9,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
0007ee1ac8edb6be588c6c8a158d2268e3228c49cf52773d168012076c118e5f,0,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
0008ef32824d3067b4d8ab70e7c79fa04c808f4d99194fe9c7827dac8bcf0b5c,1,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffeea21b8f17b728c561de93ea61e75547dd0216b36beba487c9ab3efa985fd5,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
ffeec9a34770713d0c488435fbff21db356407c98d9ddb37b535caa87979c61c,1,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
fff17a28a4594efc934648f3c33d295d452d61e2bda01df4b13952f48dfa1175,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [43]:
train_final.merge(dummy_df.groupby('customer_ID').agg('last'), on='customer_ID')

,P_2_nulls,D_39_nulls,B_1_nulls,B_2_nulls,R_1_nulls,S_3_nulls,D_41_nulls,B_3_nulls,D_42_nulls,D_43_nulls,...,D_66_1.0,D_66_None,D_68_0.0,D_68_1.0,D_68_2.0,D_68_3.0,D_68_4.0,D_68_5.0,D_68_6.0,D_68_None
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,1.000000,...,0,1,0,0,0,0,0,0,1,0
0005ef1480ffe4f0e5908eae07db8e05f2e705bca6a57db3b97d1da2eb6c7cab,0.0,0.0,0.0,0.0,0.0,0.153846,0.0,0.0,0.153846,0.000000,...,1,0,0,0,0,0,0,1,0,0
0005f2ab0cc352cd64eac9e60b981ed8c3289d20fa5fcef78973c468916fdfc9,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,0.307692,...,0,1,0,0,0,0,0,0,1,0
0007ee1ac8edb6be588c6c8a158d2268e3228c49cf52773d168012076c118e5f,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,0.000000,...,0,1,0,0,0,0,0,0,1,0
0008ef32824d3067b4d8ab70e7c79fa04c808f4d99194fe9c7827dac8bcf0b5c,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,1.000000,0.000000,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffeea21b8f17b728c561de93ea61e75547dd0216b36beba487c9ab3efa985fd5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,0.000000,...,0,1,0,0,0,0,0,0,1,0
ffeec9a34770713d0c488435fbff21db356407c98d9ddb37b535caa87979c61c,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,1.000000,...,1,0,0,0,0,0,0,0,1,0
fff17a28a4594efc934648f3c33d295d452d61e2bda01df4b13952f48dfa1175,0.0,0.0,0.0,0.0,0.0,0.538462,0.0,0.0,0.153846,1.000000,...,0,1,0,0,0,0,0,1,0,0


In [44]:
train_final = train_final.merge(dummy_df.groupby('customer_ID').agg('last'), on='customer_ID')

## Cap numerical features

Feature values are z-score normalized. I am going to say that a value greater than 3 standard deviations above or below the mean is too large, and I will cap these values to 3 and -3 respectively. The vast majority of values, and many of the min and max values, already fall within this range.

In [55]:
desc = train.drop(columns=categorical_columns).describe().T

In [66]:
desc

,count,mean,std,min,25%,50%,75%,max
P_2,466807.0,0.654822,0.244151,-3.915485e-01,0.480436,0.691194,0.862548,1.010000
D_39,470744.0,0.152576,0.270041,3.785832e-08,0.004521,0.009034,0.235852,5.356063
B_1,470744.0,0.125361,0.212780,-1.808689e+00,0.008810,0.031263,0.128594,1.324060
B_2,470556.0,0.619467,0.401543,5.485064e-08,0.103320,0.814219,1.002312,1.010000
R_1,470744.0,0.078924,0.226229,5.138723e-09,0.002898,0.005788,0.008669,2.759651
...,...,...,...,...,...,...,...,...
D_141,462354.0,0.165757,0.349234,1.073346e-08,0.003039,0.006065,0.009103,1.245575
D_142,81073.0,0.391282,0.235656,-1.394664e-02,0.201264,0.382718,0.554260,1.816997
D_143,462354.0,0.180216,0.380161,1.653580e-08,0.003035,0.006053,0.009093,1.010000
D_144,467368.0,0.053471,0.185633,1.302884e-07,0.002760,0.005524,0.008281,1.343330


In [68]:
train.columns

Index(['customer_ID', 'S_2', 'P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41',
       'B_3',
       ...
       'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143',
       'D_144', 'D_145'],
      dtype='object', length=190)

In [71]:
numerical_cols = list(train.drop(columns=categorical_columns))
numerical_cols

['customer_ID',
 'S_2',
 'P_2',
 'D_39',
 'B_1',
 'B_2',
 'R_1',
 'S_3',
 'D_41',
 'B_3',
 'D_42',
 'D_43',
 'D_44',
 'B_4',
 'D_45',
 'B_5',
 'R_2',
 'D_46',
 'D_47',
 'D_48',
 'D_49',
 'B_6',
 'B_7',
 'B_8',
 'D_50',
 'D_51',
 'B_9',
 'R_3',
 'D_52',
 'P_3',
 'B_10',
 'D_53',
 'S_5',
 'B_11',
 'S_6',
 'D_54',
 'R_4',
 'S_7',
 'B_12',
 'S_8',
 'D_55',
 'D_56',
 'B_13',
 'R_5',
 'D_58',
 'S_9',
 'B_14',
 'D_59',
 'D_60',
 'D_61',
 'B_15',
 'S_11',
 'D_62',
 'D_65',
 'B_16',
 'B_17',
 'B_18',
 'B_19',
 'B_20',
 'S_12',
 'R_6',
 'S_13',
 'B_21',
 'D_69',
 'B_22',
 'D_70',
 'D_71',
 'D_72',
 'S_15',
 'B_23',
 'D_73',
 'P_4',
 'D_74',
 'D_75',
 'D_76',
 'B_24',
 'R_7',
 'D_77',
 'B_25',
 'B_26',
 'D_78',
 'D_79',
 'R_8',
 'R_9',
 'S_16',
 'D_80',
 'R_10',
 'R_11',
 'B_27',
 'D_81',
 'D_82',
 'S_17',
 'R_12',
 'B_28',
 'R_13',
 'D_83',
 'R_14',
 'R_15',
 'D_84',
 'R_16',
 'B_29',
 'S_18',
 'D_86',
 'D_87',
 'R_17',
 'R_18',
 'D_88',
 'B_31',
 'S_19',
 'R_19',
 'B_32',
 'S_20',
 'R_20',
 'R_

In [72]:
numerical_cols.remove('customer_ID')
numerical_cols.remove('S_2')
numerical_cols

['P_2',
 'D_39',
 'B_1',
 'B_2',
 'R_1',
 'S_3',
 'D_41',
 'B_3',
 'D_42',
 'D_43',
 'D_44',
 'B_4',
 'D_45',
 'B_5',
 'R_2',
 'D_46',
 'D_47',
 'D_48',
 'D_49',
 'B_6',
 'B_7',
 'B_8',
 'D_50',
 'D_51',
 'B_9',
 'R_3',
 'D_52',
 'P_3',
 'B_10',
 'D_53',
 'S_5',
 'B_11',
 'S_6',
 'D_54',
 'R_4',
 'S_7',
 'B_12',
 'S_8',
 'D_55',
 'D_56',
 'B_13',
 'R_5',
 'D_58',
 'S_9',
 'B_14',
 'D_59',
 'D_60',
 'D_61',
 'B_15',
 'S_11',
 'D_62',
 'D_65',
 'B_16',
 'B_17',
 'B_18',
 'B_19',
 'B_20',
 'S_12',
 'R_6',
 'S_13',
 'B_21',
 'D_69',
 'B_22',
 'D_70',
 'D_71',
 'D_72',
 'S_15',
 'B_23',
 'D_73',
 'P_4',
 'D_74',
 'D_75',
 'D_76',
 'B_24',
 'R_7',
 'D_77',
 'B_25',
 'B_26',
 'D_78',
 'D_79',
 'R_8',
 'R_9',
 'S_16',
 'D_80',
 'R_10',
 'R_11',
 'B_27',
 'D_81',
 'D_82',
 'S_17',
 'R_12',
 'B_28',
 'R_13',
 'D_83',
 'R_14',
 'R_15',
 'D_84',
 'R_16',
 'B_29',
 'S_18',
 'D_86',
 'D_87',
 'R_17',
 'R_18',
 'D_88',
 'B_31',
 'S_19',
 'R_19',
 'B_32',
 'S_20',
 'R_20',
 'R_21',
 'B_33',
 'D_89',
 

In [74]:
train[numerical_cols].clip(lower=-3, upper=3)

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,NaN,NaN,...,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,NaN,NaN,...,NaN,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217
2,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,NaN,NaN,...,NaN,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603
3,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,NaN,NaN,...,NaN,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600
4,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,NaN,NaN,...,NaN,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470739,0.730505,0.589866,0.051951,1.008380,0.008432,0.137431,0.000815,0.004927,NaN,0.138725,...,NaN,NaN,NaN,0.005826,0.002661,0.006540,NaN,0.002191,0.001231,0.007120
470740,0.833949,0.005199,0.013103,1.005107,0.001183,0.134598,0.003260,0.001333,NaN,0.120524,...,NaN,NaN,NaN,0.009753,0.009238,0.006315,NaN,0.009012,0.000864,0.005220
470741,0.837949,0.008265,0.013163,1.007224,0.005757,0.133852,0.005954,0.007394,NaN,0.130741,...,NaN,NaN,NaN,0.008494,0.001622,0.009657,NaN,0.005007,0.008420,0.002532
470742,0.812465,0.002464,0.016056,1.003230,0.007137,0.136428,0.000088,0.009943,NaN,0.103616,...,NaN,NaN,NaN,0.009720,0.009944,0.001952,NaN,0.007779,0.005021,0.005594


In [76]:
for col in numerical_cols:
    train[col].clip(lower=-3, upper=3, inplace=True)

## Impute

The null count columns are scaled from 0-1. The one-hot encoded dummy columns for categorical features have values 0 and 1. The numerical features are capped between -3 and 3. I will impute -5 for null values in my numerical columns so it's noticeably different from most values but not so different that it drastically skews my min, mean, range, and potentially last features.

In [78]:
for col in numerical_cols:
    train[col].fillna(-5, inplace=True)

## Engineer aggregate features

For the numerical columns, I will calculate the min, max, median, last, first, and change values for each customer. I will only use the first value to calculate the to calculate the change (last value for the customer minus the first) and then I will drop it from the dataframe.

In [84]:
def data_preparer(df, df_labels):
    
    new_df = pd.DataFrame({'customer_ID': df['customer_ID'].unique()})
    
    for col in numerical_cols:
        mini = df.groupby('customer_ID')[col].min()
        maxi = df.groupby('customer_ID')[col].max()
        med = df.groupby('customer_ID')[col].median()
        std = df.groupby('customer_ID')[col].std()
        first = df.groupby('customer_ID')[col].first()
        last = df.groupby('customer_ID')[col].last()
        change = last - first
    
        stats = pd.DataFrame({f'{col}_min': mini, f'{col}_max': maxi,
                              f'{col}_median': med, f'{col}_std': std,
                              f'{col}_last': last, f'{col}_change': change})
    
        stats.reset_index(drop=True, inplace=True)
    
        new_df = pd.concat([new_df, stats], axis=1)
        
    new_df = new_df.merge(df_labels, on='customer_ID')
    
    new_df = new_df.merge(train_final, on='customer_ID')
    
    X = new_df.drop(columns=['customer_ID', 'target'])
    y = new_df['target']
    
    dmatrix = xgb.DMatrix(X, label=y)
    
    return dmatrix, y

In [87]:
train_x, train_y = data_preparer(train, train_labels)

## Model training

I will train an xgboost model starting with some useful hyperparameters I discovered in the tuning notebook.

In [93]:
steps = 200
seed = 42

params = {
    'verbosity': 1,
    'max_depth': 4,
    'objective': 'binary:logistic',
    'eta': 0.15,
    'random_state': seed,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.8
}

In [94]:
model = xgb.train(params, train_x, steps, early_stopping_rounds=2,
                     evals=[(train_x, 'Training')])

[0]	Training-logloss:0.60402
[1]	Training-logloss:0.53771
[2]	Training-logloss:0.48591
[3]	Training-logloss:0.44513
[4]	Training-logloss:0.41250
[5]	Training-logloss:0.38519
[6]	Training-logloss:0.36274
[7]	Training-logloss:0.34358
[8]	Training-logloss:0.32789
[9]	Training-logloss:0.31433
[10]	Training-logloss:0.30286
[11]	Training-logloss:0.29295
[12]	Training-logloss:0.28443
[13]	Training-logloss:0.27728
[14]	Training-logloss:0.27096
[15]	Training-logloss:0.26557
[16]	Training-logloss:0.26088
[17]	Training-logloss:0.25618
[18]	Training-logloss:0.25197
[19]	Training-logloss:0.24875
[20]	Training-logloss:0.24558
[21]	Training-logloss:0.24271
[22]	Training-logloss:0.24020
[23]	Training-logloss:0.23776
[24]	Training-logloss:0.23566
[25]	Training-logloss:0.23380
[26]	Training-logloss:0.23189
[27]	Training-logloss:0.23008
[28]	Training-logloss:0.22870
[29]	Training-logloss:0.22713
[30]	Training-logloss:0.22564
[31]	Training-logloss:0.22426
[32]	Training-logloss:0.22295
[33]	Training-loglos

In [97]:
def model_evaluator(model, data, y_true):
    
    y_hat = model.predict(data)
    
    y_true_final = pd.DataFrame(y_true)
    y_hat_final = pd.DataFrame(y_hat, columns=['prediction'])
    
    return myfunctions.amex_metric(y_true_final, y_hat_final)

In [98]:
result = model_evaluator(model, train_x, train_y)

In [99]:
result

0.9340109782806754